In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
with open(r"C:\Users\tom_r\Desktop\Machine-Learning\RF_spectrum_analysis\subset.pkl", 'rb') as file:
    dataset = pickle.load(file)

In [4]:
for k, v in dataset.items():
    print(f"{k}: shape {v.shape}, dtype {v.dtype}")

X: shape (532480, 1024, 2), dtype float16
y: shape (532480,), dtype int64
snr: shape (532480,), dtype int64


In [5]:
unique_labels = np.unique(dataset['y'])
label_map = {old: new for new, old in enumerate(unique_labels)}

# Remap y labels
mapped_y = np.vectorize(label_map.get)(dataset['y'])

In [6]:
mapped_y

array([0, 0, 0, ..., 4, 4, 4], shape=(532480,))

In [7]:
X_tensor = torch.tensor(dataset['X'], dtype=torch.float32)  
X_tensor = X_tensor.permute(0, 2, 1)  # (N, 2, 1024)

In [35]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3)
        self.batchnorm1 = nn.BatchNorm1d(16)
        self.conv_layer2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.batchnorm2 = nn.BatchNorm1d(32)
        self.max_pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv_layer3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.conv_layer4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.batchnorm4 = nn.BatchNorm1d(128)
        self.max_pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(32384, 128)
        self.dropout = nn.Dropout(p=0.5) 
        self.fc2 = nn.Linear(128, 5)  # 5 classes

    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)

        x = self.conv_layer2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = self.max_pool1(x)

        x = self.conv_layer3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)

        x = self.conv_layer4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = self.max_pool2(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)      
        x = self.fc2(x)

        return x


In [25]:
m = CNN()
output = m.forward(X_tensor[0].unsqueeze(0))  # now shape: (1, 2, 1024)
output.shape

torch.Size([1, 32384])


torch.Size([1, 5])

In [13]:
class RFDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).permute(0, 2, 1)  # (N, 2, 1024)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
rf_dataset = RFDataset(dataset['X'], mapped_y)
rf_loader = DataLoader(rf_dataset, batch_size=254, shuffle=True)

In [29]:
for batch_X, _ in rf_loader:
    print(batch_X.mean(), batch_X.std())
    break


tensor(-0.0002) tensor(0.7070)


In [15]:
# calculate steps per epoch for training
trainSteps = len(rf_loader.dataset) // 64
trainSteps

8320

In [36]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5 

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (batch_X, batch_y) in enumerate(rf_loader):
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == batch_y).sum().item()
        total += batch_y.size(0)

        # Print progress every 100 batches
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(rf_loader)}], Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(rf_loader)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1} finished → Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch [1/5], Step [100/2097], Loss: 1.0895
Epoch [1/5], Step [200/2097], Loss: 0.8228
Epoch [1/5], Step [300/2097], Loss: 0.8251
Epoch [1/5], Step [400/2097], Loss: 0.7457
Epoch [1/5], Step [500/2097], Loss: 0.6581
Epoch [1/5], Step [600/2097], Loss: 0.6798
Epoch [1/5], Step [700/2097], Loss: 0.7023
Epoch [1/5], Step [800/2097], Loss: 0.6407
Epoch [1/5], Step [900/2097], Loss: 0.6100
Epoch [1/5], Step [1000/2097], Loss: 0.6470
Epoch [1/5], Step [1100/2097], Loss: 0.6483
Epoch [1/5], Step [1200/2097], Loss: 0.5374
Epoch [1/5], Step [1300/2097], Loss: 0.5570
Epoch [1/5], Step [1400/2097], Loss: 0.6118
Epoch [1/5], Step [1500/2097], Loss: 0.6180
Epoch [1/5], Step [1600/2097], Loss: 0.5264
Epoch [1/5], Step [1700/2097], Loss: 0.5624
Epoch [1/5], Step [1800/2097], Loss: 0.5452
Epoch [1/5], Step [1900/2097], Loss: 0.6152
Epoch [1/5], Step [2000/2097], Loss: 0.6532
Epoch 1 finished → Avg Loss: 0.6798, Accuracy: 69.14%
Epoch [2/5], Step [100/2097], Loss: 0.5906
Epoch [2/5], Step [200/2097], Lo